# Lab 8. Apache Airflow - część 1.

Apache Airflow® to platforma open-source do opracowywania, harmonogramowania i monitorowania zadań wsadowych. Dzięki językowy Python, w którym definiowane są grafy zadań możliwe jest połączenie tych definicji praktycznie z każdą technologią. Apache Airflow pozwala na tworzenie zadań o różnym stopniu skomplikowania a same zadania pisane są w postaci kodu w języku Python. Apache Airflow dostarcza również rozbudowany interfejs graficzny, dzięki któremu możemy śledzić wykonanie zadań oraz przepływy zdefiniowane w każdym zadaniu.

Apache Airflow nie został stworzony do uruchamiania zadań, których wykonanie trwa w nieskończoność.

## 1. Instalacja i uruchomienie Apache Airflow

W opisywanym przypadku instalacja Apache Airflow zostanie wykonana za pośrednictwem menedżera pip, który aktualnie jest jedynym oficjalnie wspieranym rozwiązaniem do instalacji tego modułu języka Python.


Polcecenie:
```bash
pip install apache-airflow
```

Instalacja i uruchomienie przedstawione w tym labie nie są odpowiednie dla środowiska produkcyjnego, co jest jasno zakomunikowane w dokumentacji.

> UWAGA! Wykonanie poniższych komend zalecane jest w nowym oknie terminala, gdzie można śledzić kolejne komunikaty wyświetlane w tym oknie, ale również ze względu na to, że spowoduje to uruchomienie usługi, która będzie działała póki jej nie przerwiemy i nie będziemy mogli w tym oknie podawać kolejnych komend, bez przełączania między zadaniami w tle.

Uruchomienie zainstalowanego modułu Apache Airflow można wykonać w trybie `standalone`.

Polecenie
```console
airflow standalone 
```

uruchomi kilka modułów z domyślnymi parametrami.
Gdybyśmy chcieli uruchomić je inaczej możemy wywołać kolejne moduły jak podano w dokumentacji:

```bash
airflow db migrate

airflow users create \
    --username admin \
    --firstname Peter \
    --lastname Parker \
    --role Admin \
    --email spiderman@superhero.org

airflow webserver --port 8080 # może się przydać, jeżeli w naszym kontenerze działa już jakaś usługa na tym porcie

airflow scheduler
```


Więcej informacji o procesie uruchomienia Apache Airflow można znaleźć tutaj:
https://airflow.apache.org/docs/apache-airflow/stable/start.html

Po uruchomieniu serwera obserwuj komunikaty w konsoli, gdyż w pewnym momencie pojawi się informacja o utworzonym koncie razem z hasłem do zalogowania do serwisu www dostępnego pod adresem http://localhost:9090 (takie domyślne mapowanie zostało stworzone w pliku konfiguracyjnym 9090:8080).

Po zalogowaniu powinno pojawić się okno ze wszystkimi zdefiniowanymi grafami z załadowanej lokalizacji (która jest określona w pliku konfiguracyjnym, opisanym w dalszej części tych materiałów).

![Apache Aiflow](airflow_1.png)

## 2. Definiowanie zadań.

Omówienie przykładowej definicji zadania opiera się o zadanie z dokumentacji Apache Airflow (`airflow/example_dags/tutorial.py`).

Włączenie widoczności numerów linii skrótem `SHIFT + L`.

_**Listing 1**_

In [ ]:
import textwrap
from datetime import datetime, timedelta

# The DAG object; we'll need this to instantiate a DAG
from airflow.models.dag import DAG

# Operators; we need this to operate!
from airflow.operators.bash import BashOperator

with DAG(
    "tutorial",
    # These args will get passed on to each operator
    # You can override them on a per-task basis during operator initialization
    default_args={
        "depends_on_past": False,
        "email": ["airflow@example.com"],
        "email_on_failure": False,
        "email_on_retry": False,
        "retries": 1,
        "retry_delay": timedelta(minutes=5),
        # 'queue': 'bash_queue',
        # 'pool': 'backfill',
        # 'priority_weight': 10,
        # 'end_date': datetime(2016, 1, 1),
        # 'wait_for_downstream': False,
        # 'sla': timedelta(hours=2),
        # 'execution_timeout': timedelta(seconds=300),
        # 'on_failure_callback': some_function, # or list of functions
        # 'on_success_callback': some_other_function, # or list of functions
        # 'on_retry_callback': another_function, # or list of functions
        # 'sla_miss_callback': yet_another_function, # or list of functions
        # 'on_skipped_callback': another_function, #or list of functions
        # 'trigger_rule': 'all_success'
    },
    description="A simple tutorial DAG",
    schedule=timedelta(days=1),
    start_date=datetime(2021, 1, 1),
    catchup=False,
    tags=["example"],
) as dag:

    # t1, t2 and t3 are examples of tasks created by instantiating operators
    t1 = BashOperator(
        task_id="print_date",
        bash_command="date",
    )

    t2 = BashOperator(
        task_id="sleep",
        depends_on_past=False,
        bash_command="sleep 5",
        retries=3,
    )
    t1.doc_md = textwrap.dedent(
        """\
    #### Task Documentation
    You can document your task using the attributes `doc_md` (markdown),
    `doc` (plain text), `doc_rst`, `doc_json`, `doc_yaml` which gets
    rendered in the UI's Task Instance Details page.
    ![img](https://imgs.xkcd.com/comics/fixing_problems.png)
    **Image Credit:** Randall Munroe, [XKCD](https://xkcd.com/license.html)
    """
    )

    dag.doc_md = __doc__  # providing that you have a docstring at the beginning of the DAG; OR
    dag.doc_md = """
    This is a documentation placed anywhere
    """  # otherwise, type it like this
    templated_command = textwrap.dedent(
        """
    {% for i in range(5) %}
        echo "{{ ds }}"
        echo "{{ macros.ds_add(ds, 7)}}"
    {% endfor %}
    """
    )

    t3 = BashOperator(
        task_id="templated",
        depends_on_past=False,
        bash_command=templated_command,
    )

    t1 >> [t2, t3]

We fragmencie:

```python
with DAG(
    "tutorial",
    # These args will get passed on to each operator
    # You can override them on a per-task basis during operator initialization
    default_args={
        "depends_on_past": False,
        "email": ["airflow@example.com"],
        "email_on_failure": False,
        "email_on_retry": False,
        "retries": 1,
        "retry_delay": timedelta(minutes=5),
        # 'queue': 'bash_queue',
        # 'pool': 'backfill',
        # 'priority_weight': 10,
        # 'end_date': datetime(2016, 1, 1),
        # 'wait_for_downstream': False,
        # 'sla': timedelta(hours=2),
        # 'execution_timeout': timedelta(seconds=300),
        # 'on_failure_callback': some_function, # or list of functions
        # 'on_success_callback': some_other_function, # or list of functions
        # 'on_retry_callback': another_function, # or list of functions
        # 'sla_miss_callback': yet_another_function, # or list of functions
        # 'on_skipped_callback': another_function, #or list of functions
        # 'trigger_rule': 'all_success'
    },
    description="A simple tutorial DAG",
    schedule=timedelta(days=1),
    start_date=datetime(2021, 1, 1),
    catchup=False,
    tags=["example"],
) as dag:
```
zdefiniowano identyfikator tego DAGa (`dag_id`) o nazwie `tutorial` (który można zdefiniować również przez kwargs `dag_id="tutorial"`) oraz
zdefiniowane zostały parametry, które zostaną przekazane do każdego operatora, które w ramach tego grafu zadań zostaną zdefiniowane. Można je zapisać jako domyślne do późniejszego wykorzystania i ewentualnego nadpisania tylko niektórych z nich.

Kolejno zdefiniowany jest opis oraz interwał wykonania zadania (`timedelta`), tutaj 1 dzień, datę pierwszego uruchomienia, `catchup` (zobacz: https://airflow.apache.org/docs/apache-airflow/stable/core-concepts/dag-run.html#catchup) oraz tagi.


> Dokumentacja obiektów DAG: https://airflow.apache.org/docs/apache-airflow/stable/core-concepts/dags.html
> 
> Lista i opis wszystkich parametrów dla obiektu `BaseOperator`, z którego dziedziczą
> wszystkie inne operatory znajduje się pod adresem:
> https://airflow.apache.org/docs/apache-airflow/stable/_api/airflow/models/baseoperator/index.html#airflow.models.baseoperator.BaseOperator

**Operatory** do zdefiniowane szablony zadań, które można wykorzystać w grafie zadań lub można rozszerzyć je o własne implementacje. Listę zdefiniowanych operatorów oraz ich dokumentację znajdziemy tu: https://airflow.apache.org/docs/apache-airflow/stable/core-concepts/operators.html#operators


Wykorzystanie operatorów polega na zdefiniowaniu zadań (tasków), które te operatory wykorzystują, jak poniższy fragment:

```python
# t1, t2 and t3 are examples of tasks created by instantiating operators
t1 = BashOperator(
    task_id="print_date",
    bash_command="date",
)

t2 = BashOperator(
    task_id="sleep",
    depends_on_past=False,
    bash_command="sleep 5",
    retries=3,
)
t3 = BashOperator(
    task_id="templated",
    depends_on_past=False,
    bash_command=templated_command,
)
```

Argument `bash_command` operatora `BashOperator` może również wskazywać na skrypt powłoki, a ścieżka do niego przekazywana jest relatywnie do położenia skryptu samej definicji grafu (czyli tutaj tutorial.py).

Widać też w przykładzie zdefiniowaną dokumentację (`dag.doc_md` oraz `t1.doc_md`), która będzie również widoczna w graficznym interfejsie obsługi Apache Airflow, ale już w samym przykładzie widać, że możemy to również zrobić określając tę dokumentację jako `doc` (plain text), `doc_rst`, `doc_json` czy `doc_yaml`.

I na sam koniec określone są zależności między zadaniami zdefiniowanymi w ramach tego grafu.

```python
t1 >> [t2, t3]
```

Powyższy zapis oznacza, że zadania 2 oraz 3 zależą od zadania 1.

Rozwinięcie nazwy DAG oznacza Directed Acyclic Graphs czyli skierowane acykliczne grafy i jeżeli zdefiniujemy w naszym grafach cykle Apache Airflow zgłosi wyjątek przy próbie uruchomienia takiego grafu. Wyjątek pojawi się również w przypadku gdy pojawi się odwołanie do zależności więcej niż jeden raz.

Z dokumentacji możemy również zobaczyć inne przykłady definiowania zależności, np. takie jak poniżej.

```python

# 1. Zadanie 2 zależy od pomyslnego wykonania zadania 1
t1.set_downstream(t2)
# lub
t1 >> t2
# lub
t2.set_upstream(t1)
# lub
t2 << t1


# 2. łańcuch zależności
t1 >> t2 >> t3

# 3. można również określić listę zależnych zadań jak poniżej
t1.set_downstream([t2, t3])
t1 >> [t2, t3]
[t2, t3] << t1
```

## 3. Konfiguracja Apache Airflow i jej zachowanie w naszym środowisku

Konfiguracja narzędzia Apache Airflow zostanie domyślnie stworzona w folderze `~/airflow` czyli w naszym przypadku jeżeli korzystamy ze środowiska docker skonfigurowanego w trakcie zajęć będzie to `/home/spark/airflow/`, które póki co nie będzie utrwalane na dysku maszyny hosta gdyż mapowanie w `volumnes` zostało ustawione tylko na folder `/opt/spark/work-dir`. Abyśmy mogli przechowywać konfigurację Aiflow między uruchomieniami kontenera oraz aby meta baza Airflow również była utrwalana należy dodać odpowiednią konfigurację do pliku `docker-compose.yml`. Możemy to zrobić jak poniżej:

```yml
...
    volumes:
       - ./apps:/opt/spark/work-dir
       - ./airflow:/home/spark/airflow
```

Pamiętajmy jednak, że te ustawienia zostaną wykorzystane dopiero po zatrzymaniu kontenera i ponownym jego uruchomieniu, więc na pewno możemy stracić to co już w ustawieniach Airflow jest w tym momencie. Możemy to jednak skopiować z poziomu kontenera przed jego zatrzymaniem, np tak:

```bash
# będąc w folderze /home/spark
cp -R ~/airflow /opt/spark/work-dir
```

Pamiętajmy, że najlepiej wykonać to po zatrzymaniu Apache Aiflow, a przed zatrzymaniem kontenera. Wysłanie komendy zakończenia wykonania bieżącego zadania uruchomionego w terminalu można wykonać za pomocą kombinacji klawiszy `CTRL + c` - trzeba tutaj poczekać dłuższą chwilę, aż usługa się zatrzyma.

Po zatrzymaniu Apache Airflow, wykonaniu kopii i zatrzymaniu kontenera, powinniśmy przenieść skopiowany folder aiflow z folderu hosta (powinien znajdować się w folderze `apps`) na ten sam poziom co folder `apps`. Teraz możemy wywołać polecenie `docker compose up` w terminalu ze ścieżki, w której znajduje się zmodyfikowana wersja pliku `docker-compose.yml` i folder powinien się mapować teraz przy kolejnych uruchomieniach kontenera.

## Zadania

**Zadanie 1**  
Zainstaluj i uruchom Apache Airflow. Zaloguj się do panelu aplikacji www.

**Zadanie 2**  
Uruchom zadanie `airflow/example_dags/tutorial.py` i prześledź w aplikacji jego wykonanie, log, wykres Gannt i inne statystyki, które są udostępniane przez interfejs. Na zakładce `Graph` po zaznaczeniu konkretnego zadania z grafu zmienia się ścieżka powrotu na górze okna i przechodząc teraz po zakładkach widzimy opis i inne właściwości zdefiniowane dla zadania, a nie całego grafu. Przeglądnij te informacje dla trzech zdefiniowanych zadań w tym grafie.

**Zadanie 3**  
Stwórz nowy graf poprzez modyfikację grafu z listingu 1 i zarejestruj go pod nazwą `tutorial_hourly` w pliku o nazwie `tutorial_hourly.py` i zapisz w tym samym folderze co oryginał. W definicji tego grafu zmodyfikuj:
* interwał uruchamiania z dziennego na godzinny,
* opis,
* dokumentację obiektu dag (widoczną później również poprzez interfejs graficzny),
* datę pierwszego uruchomienia na bieżącą,
* tag (np. lab8).

**Zadanie 4**  

Posługując się opisem w labie wykonaj operacje, które pozwolą na przechowanie konfiguracji Apache Airflow pomiędzy uruchomieniami kontenera.